<a href="https://colab.research.google.com/github/melanieyes/sentence-transformer/blob/main/Vietnamese_Legal_Document_Retrieval_Using_Sentence_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Vietnamese legal document retrieval using Sentence Transformers**

In [ ]:
!pip install -q mteb transformers==4.45.2 sentence-transformers==3.1.1 wandb

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import BinaryClassificationEvaluator, EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from datasets import load_dataset
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
import pandas as pd
from ast import literal_eval
import torch


os.environ['WANDB_DISABLED'] = 'true'


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
is_word = False
train_batch_size = 32
num_epochs = 1
model_save_path = os.path.join(os.getcwd(), 'output', 'model')
top_k = 32

## Load and Prepare Vietnamese legal documents dataset

In [ ]:
!wget https://huggingface.co/datasets/tmnam20/BKAI-Legal-Retrieval/resolve/main/archive.zip
!unzip -o -qq archive.zip -d data

--2024-12-18 04:13:56--  https://huggingface.co/datasets/tmnam20/BKAI-Legal-Retrieval/resolve/main/archive.zip
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/77/34/7734ba2101c276edd0fc8c24cc368330a94a50e26dadb6b459834f0603785f20/35bef18231742a2ec66889a9074a8283b0807448b777bf8dda45a95859d0e875?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27archive.zip%3B+filename%3D%22archive.zip%22%3B&response-content-type=application%2Fzip&Expires=1734754437&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDc1NDQzN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzc3LzM0Lzc3MzRiYTIxMDFjMjc2ZWRkMGZjOGMyNGNjMzY4MzMwYTk0YTUwZTI2ZGFkYjZiNDU5ODM0ZjA2MDM3ODVmMjAvMzViZWYxODIzMTc0MmEyZWM2Njg4OWE5MDc0YTgyODNiMDgwNzQ0OGI3Nz

In [ ]:
corpus_data = pd.read_csv('data/corpus.csv')
train_data = pd.read_csv('data/train_split.csv', converters={'context': literal_eval})
test_data = pd.read_csv('data/val_split.csv', converters={'context': literal_eval})

In [ ]:
corpus_data.head()

,text,cid
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0
1,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",1
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê\n...",3
4,"Phù hiệu của lực lượng tuần tra, canh gác đê\n...",4


In [ ]:
train_data['cid'] = train_data['cid'].apply(lambda x: [int(i) for i in x[1:-1].split()])
train_data.head()

,question,context,cid,qid
0,Liên đoàn Luật sư Việt Nam là tổ chức xã hội –...,[“Điều 2. Địa vị pháp lý của Liên đoàn Luật sư...,[142820],72600
1,Tên hợp tác xã bị rơi vào trường hợp cấm thì c...,"[Tên hợp tác xã, liên hiệp hợp tác xã\n1. Tên ...","[27817, 72117]",147562
2,Tài xế lái xe ô tô khách 50 chỗ ngồi bao lâu t...,"[""1. Sử dụng lái xe bảo đảm sức khỏe theo tiêu...","[33215, 56201]",142107
3,Các bước chuẩn bị thủ thuật bó bột Cravate sẽ ...,[BỘT CRAVATE\n...\nIV. CHUẨN BỊ\n1. Người thực...,[148158],77353
4,Viên chức Hộ sinh hạng 4 có những nhiệm vụ gì ...,[Hộ sinh hạng IV - Mã số: V.08.06.16\n1. Nhiệm...,[188132],113090


In [ ]:
test_data['cid'] = test_data['cid'].apply(lambda x: [int(i) for i in x[1:-1].split()])
test_data.head()

,question,context,cid,qid
0,Phó Tổng Giám đốc Ngân hàng Chính sách xã hội ...,[Áp dụng chế độ tiền lương và phụ cấp quy định...,[140864],70867
1,Ai có thẩm quyền quyết định thành lập Hội đồng...,[Thành lập Hội đồng\n1. Bộ trưởng Bộ Y tế ra q...,[62339],813
2,Thời hiệu xử phạt đối với nhà xuất bản thực hi...,[Mức phạt tiền và thẩm quyền xử phạt\n...\n4. ...,[63171],40392
3,"Việc ký kết, thực hiện thỏa thuận quốc tế nhân...",[Báo cáo tình hình ký kết và thực hiện thỏa th...,[157761],85946
4,Đề án sử dụng tài sản công tại đơn vị sự nghiệ...,"[""Điều 44. Đề án sử dụng tài sản công tại đơn ...",[95397],55607


In [ ]:
samples = {'train': [], 'test': []}
data = {'train': train_data, 'test': test_data}

for subset in ['train', 'test']:
    for i, row in data[subset].iterrows():
        question = row['question']
        context = row['context']
        for c in context:
            samples[subset].append(InputExample(texts=[question, c]))

In [ ]:
print(f"Train size: {len(samples['train'])}")
print(f"Test size: {len(samples['test'])}")

Train size: 89592
Test size: 29864


In [ ]:
print(f'Training Example: {samples["train"][0].texts}')

Training Example: ['Liên đoàn Luật sư Việt Nam là tổ chức xã hội – nghề nghiệp có tư cách pháp nhân, có con dấu, tài khoản riêng?', '“Điều 2. Địa vị pháp lý của Liên đoàn Luật sư Việt Nam\n1. Liên đoàn Luật sư Việt Nam là tổ chức xã hội - nghề nghiệp thống nhất trong toàn quốc của các Đoàn Luật sư, các luật sư Việt Nam; có tư cách pháp nhân, có con dấu, tài khoản.\n2. Biểu tượng của Liên đoàn Luật sư Việt Nam là hình tròn nền xanh da trời, chính giữa là cán cân công lý gắn với hình tượng cuốn sách, dưới cán cân công lý là dòng chữ “VIETNAM BAR FEDERATION", hai bên mỗi bên có ba dải màu vàng đậm, phía trên là ngôi sao vàng hình cờ Tổ quốc Việt Nam và dòng chữ Liên đoàn Luật sư Việt Nam.\n3. Tên giao dịch quốc tế của Liên đoàn Luật sư Việt Nam là Vietnam Bar Federation (viết tắt là VBF).\n4. Trụ sở của Liên đoàn Luật sư Việt Nam đặt tại Hà Nội – Thủ đô nước Cộng hoà xã hội chủ nghĩa Việt Nam.”']


In [ ]:
train_dataloader = DataLoader(samples['train'], shuffle=True, batch_size=train_batch_size)

## Load pre-trained Sentence Transformers model

In [ ]:
model_id = "google-bert/bert-base-multilingual-cased"
word_embedding_model = models.Transformer(model_id, max_seq_length=512, cache_dir="./cache")
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=False,
    pooling_mode_cls_token=True,
    pooling_mode_max_tokens=False,
)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device='cuda', cache_folder='./cache')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

## Train the embedding model

In [ ]:
train_loss = losses.CachedMultipleNegativesRankingLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    output_path=model_save_path,
    optimizer_params={'lr': 3e-5},
)

In [ ]:
model.save(model_save_path)

## Evaluate the model with MTEB on the BKAI Legal Document Retrieval dataset

In [ ]:
from mteb import MTEB
from mteb.abstasks.AbsTaskRetrieval import AbsTaskRetrieval
from mteb.abstasks.TaskMetadata import TaskMetadata

class BKAILegalDocRetrievalTask(AbsTaskRetrieval):
    metadata = TaskMetadata(
            name="BKAILegalDocRetrieval",
            description="",
            reference="https://github.com/embeddings-benchmark/mteb/blob/main/docs/adding_a_dataset.md",
            type="Retrieval",
            category="s2p",
            modalities=["text"],
            eval_splits=["test"],
            eval_langs=["vi"],
            main_score="ndcg_at_10",
            dataset={
                "path": "data",
                "revision": "d4c5a8ba10ae71224752c727094ac4c46947fa29",
            },
            date=("2012-01-01", "2020-01-01"),
            form="Written",
            domains=["Academic", "Non-fiction"],
            task_subtypes=["Scientific Reranking"],
            license="cc-by-nc-4.0",
            annotations_creators="derived",
            dialect=[],
            text_creation="found",
            bibtex_citation="",
        )

    data_loaded = True

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        global corpus_data, data

        self.corpus = {}
        self.queries = {}
        self.relevant_docs = {}

        shared_corpus = {}
        for _, row in corpus_data.iterrows():
            cid_str = f'c{row["cid"]}'
            shared_corpus[cid_str] = {"text": row['text'], "_id": row['cid']}

        for split in data:
            self.corpus[split] = shared_corpus
            self.queries[split] = {}
            self.relevant_docs[split] = {}

        for split in data:
            for i, row in data[split].iterrows():
                qid, cids = row['qid'], row['cid']
                question = row['question']
                qid_str, cids_str = f'q{qid}', [f'c{cid}' for cid in cids]

                self.queries[split][qid_str] = question

                for cid_str in cids_str:
                    if cid_str not in self.relevant_docs[split]:
                        self.relevant_docs[split][qid_str] = {}
                    self.relevant_docs[split][qid_str][cid_str] = 1

        self.data_loaded = True

In [ ]:
custom_task = BKAILegalDocRetrievalTask()

In [ ]:
evaluation = MTEB(tasks=[custom_task])
evaluation.run(model)

## Retrieve

In [ ]:
passages = corpus_data['text'].tolist()
print(f'Number of passages: {len(passages)}')

Total number of passages: 10


In [ ]:
corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    print("Input question:", query)

    ##### Semantic Search #####
    # Encode the query using the embedding and find potentially relevant passages
    question_embedding = model.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    # Output of top-5 hits from embedding model
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:3]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    return hits

In [ ]:
search(query = "Lực lượng tuần tra, canh gác đê được trang bị những gì")

Input question: Lực lượng tuần tra, canh gác đê được trang bị những gì

-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.999	Phù hiệu của lực lượng tuần tra, canh gác đê Phù hiệu của lực lượng tuần tra, canh gác đê là một băng đỏ rộng 10cm, có ký hiệu “KTĐ” màu vàng. Phù hiệu được đeo trên khuỷu tay áo bên trái, chữ “KTĐ” hướng ra phía ngoài.
	0.999	Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.
	0.999	1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê). 2. Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định

[{'corpus_id': 4, 'score': 0.9992641806602478},
 {'corpus_id': 0, 'score': 0.9990738034248352},
 {'corpus_id': 1, 'score': 0.9990490078926086},
 {'corpus_id': 9, 'score': 0.9990201592445374},
 {'corpus_id': 3, 'score': 0.9987598657608032},
 {'corpus_id': 7, 'score': 0.9964682459831238},
 {'corpus_id': 8, 'score': 0.9956154823303223},
 {'corpus_id': 6, 'score': 0.9946563243865967},
 {'corpus_id': 2, 'score': 0.9944443106651306},
 {'corpus_id': 5, 'score': 0.9936413764953613}]